In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from math import pi
import time 
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import IBMQ,Aer

import Qconfig_IBMQ_experience
IBMQ.save_account(Qconfig_IBMQ_experience.APItoken, overwrite=True)
IBMQ.load_account()
my_provider = IBMQ.get_provider()
my_provider.backends()


C:\Users\Arseny\anaconda3\lib\site-packages\qiskit\providers\ibmq\ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [2]:
my_provider = IBMQ.get_provider()
backend  = my_provider.get_backend('ibmqx2')
#this one compares accuracy for real quantum computers

In [3]:
q = QuantumRegister(1)
c = ClassicalRegister(1)
qc = QuantumCircuit(q, c)
qh = QuantumRegister(1)
ch = ClassicalRegister(1)
qch = QuantumCircuit(qh, ch)#double up for measurement of both theta and phi

In [4]:
theta = pi/2-pi/3
phi = pi/3
qc.h(0)
qc.u1(-2*theta,0)
qc.h(0)
qc.u1(phi-pi/2,0)
qch.h(0)
qch.u1(-2*theta,0)
qch.h(0)
qch.u1(phi-pi/2,0)
qch.h(0)
qc.measure(q, c)
qc.draw()

┌───┐┌───────────┐┌───┐┌───────────┐┌─┐
q0_0: ┤ H ├┤ U1(-pi/3) ├┤ H ├┤ U1(-pi/6) ├┤M├
      └───┘└───────────┘└───┘└───────────┘└╥┘
c0: 1/═════════════════════════════════════╩═
                                           0

In [5]:
number_of_shots = 1024
job = execute(qc, backend, shots=number_of_shots)

result = job.result()
counts = result.get_counts(qc)
zeros = counts['0']/number_of_shots
theta_new = np.arcsin(np.sqrt(zeros)) #theta from our method
print(theta_new/np.pi) 

0.3248454393902103


In [6]:
qch.measure(qh, ch)
qch.draw()

┌───┐┌───────────┐┌───┐┌───────────┐┌───┐┌─┐
q1_0: ┤ H ├┤ U1(-pi/3) ├┤ H ├┤ U1(-pi/6) ├┤ H ├┤M├
      └───┘└───────────┘└───┘└───────────┘└───┘└╥┘
c1: 1/══════════════════════════════════════════╩═
                                                0

In [7]:
number_of_shots = 1024
job = execute(qch, backend, shots=number_of_shots)

result = job.result()
counts = result.get_counts(qch)
zeros = counts['0']/number_of_shots # the result
cosine = (zeros * 2 - 1) / np.sin(2*theta)
angle = np.arccos(cosine) #phi from our method
print (angle/np.pi) 

0.3305930892136006


In [8]:
q2 = QuantumRegister(1)
c2 = ClassicalRegister(1)
qc2 = QuantumCircuit(q2, c2)
qc2.u3(pi-2*pi/3, pi/3, 0, 0)
qc2.measure(q2, c2)
q2h = QuantumRegister(1)
c2h = ClassicalRegister(1)
qc2h = QuantumCircuit(q2h, c2h)
qc2h.u3(pi-2*pi/3, pi/3, 0, 0)
qc2h.h(0)
qc2h.measure(q2h, c2h)#double up for measurement of both theta and phi
number_of_shots = 1024
job = execute(qc2, backend, shots=number_of_shots)

result = job.result()
counts = result.get_counts(qc2)
zeros = counts['0']/number_of_shots
theta_new_u3 = np.arcsin(np.sqrt(zeros))
print(theta_new_u3/np.pi) #theta from u3

0.32870533200649266


In [9]:
number_of_shots = 1024
job = execute(qc2h, backend, shots=number_of_shots)

result = job.result()
counts = result.get_counts(qc2h)
zeros = counts['0']/number_of_shots # the result
cosine = (zeros * 2 - 1) / np.sin(2*theta)
angle_u3 = np.arccos(cosine) 
print (angle_u3/np.pi) #phi from u3

0.32557092908970076


In [10]:
theta_custom_error = np.abs(theta_new - pi/3)/(pi/3)
theta_u3_error = np.abs(theta_new_u3 - pi/3)/(pi/3)
print(theta_custom_error,' ',theta_u3_error) #comparision between relative errors for theta

0.02546368182936889   0.01388400398052202


In [11]:
phi_custom_error = np.abs(angle - pi/3)/(pi/3)
phi_u3_error = np.abs(angle_u3 - pi/3)/(pi/3)
print(phi_custom_error,' ',phi_u3_error) #comparision between relative errors for phi

0.008220732359198098   0.023287212730897587
